**Setup pre-requisites for making API calls to SureMDM**

In [ ]:
import requests
import json
import base64
import os

#SureMDM API URL of your account. Replace this with your account URL
url = "https://licensedindia.in.suremdm.io/api/"
#Add headers
headers = {
    #Api-Key header, Replace with your API KEY
    'ApiKey': "APIKEY-XYZ",
    #Set content type
    'Content-Type': "application/json",
    }
#Basic authentication credentials
Credentials=("emailid","password") #Replace with your username and password

#Device

##List Device

**API to List all the Devices**

This api allows you to list all the devices on the basis of GroupID and store all the listed deviceids in a variable DeviceIds

Request Body Schema

| Parameter                       | Data Type | Description                                                |
|---------------------------|-----------|------------------------------------------------------------|
| AdanceSearch              | Boolean   | Enable advanced search (true/false).                       |
| AdvSearchJobID            | String    | Advanced search job ID.                                    |
| AdvSearchValue            | List      | Values for advanced search.                                |
| SearchColumns             | List      | Columns to be searched.                                   |
| EnableDeviceGlobalSearch  | Boolean   | Enable global search for devices (true/false).            |
| ID                        | Value     | Group ID for filtering devices.                           |
| IsSearch                  | Boolean   | Indicate if it's a search operation (true/false).         |
| IsTag                     | Boolean   | Indicate if tag use for filteration (true/false).            |
| Limit                     | Integer   | Maximum number of results to retrieve.                    |
| Offset                    | Integer   | Number of results to skip before retrieving.              |
| SearchValue               | String    | Search value for device search.                           |
| SortColumn                | String    | Column for sorting results.                               |
| SortOrder                 | String    | Sort order (asc/desc) for sorting results.                |
| Columns                   | List      | Columns to include in the response.                       |
| IsIncludedBlackListed     | Boolean   | Include blacklisted devices (true/false).                |


In [ ]:
GroupID = "null"  # Enter the Group ID  #GroupID =>  null for home group
Body = f'''{{
    "AdanceSearch": false,
    "AdvSearchJobID": "",
    "AdvSearchValue": [],
    "SearchColumns": [],
    "EnableDeviceGlobalSearch": false,
    "ID":null,
    "IsSearch": false,
    "IsTag": false,
    "Limit": 100,
    "Offset": 0,
    "SearchValue": "",
    "SortColumn": "DeviceName",
    "SortOrder": "asc",
    "Columns": [
        "DeviceName",
        "DeviceModelName",
        "ConnectionStatus",
        "LastTimeStamp",
        "Battery",
        "DeviceTimeStamp",
        "ReleaseVersion"
    ],
    "IsIncludedBlackListed": false
}}'''

#Limit:maximum number of results (rows) you want to receive in the response
#Offset=0 =>start retrieving results from the beginning, Offset=10 =>skip the first 10 results and start retrieving results from the 11th one and so on


#Send request
response = requests.post(url+"devicegrid",auth=Credentials,data=Body,headers=headers)

print(json.dumps(json.loads(response.text), indent=4))
response_data = response.json()
print(response.text)

if "rows" in response_data and len(response_data["rows"]) > 0:
    DeviceIds = [device["DeviceID"] for device in response_data["rows"]]
    print("Device IDs:", DeviceIds)


{
    "rows": [
        {
            "DeviceID": "84f35c6b-3e8c-4c2f-b983-82943747425b",
            "ConnectionStatus": "Offline",
            "GroupID": "null",
            "DeviceIPAddress": "182.78.143.78",
            "DeviceName": "100",
            "LastTimeStamp": "2023-09-01T09:37:03.399Z",
            "DeviceModelName": "SM-M045F",
            "PlatformType": "Android",
            "Operator": "",
            "Battery": 5,
            "PhoneSignal": 0,
            "AgentVersion": 27.28016,
            "SureLockVersion": -1,
            "SureFoxVersion": -1,
            "SureVideoVersion": -1,
            "AgentFormatVersion": "27.28.16",
            "SureLockFormatVersion": "-1",
            "SureFoxFormatVersion": "-1",
            "SureVideoFormatVersion": "-1",
            "DefaultHome": "com.sec.android.app.launcher",
            "IMEI": "N/A",
            "IMSI": "N/A",
            "PhoneNumber": "",
            "SerialNumber": "N/A",
            "TrackingOn": 0,
      

#Group

##Create Group

**API to create new group**

This api allows you to create new Group

Request Body Schema

| Parameter           | Data Type | Description                              |
|---------------|-----------|------------------------------------------|
| GroupID       | String     | ID of the parent group       |
| GroupName     | String    | Name of the group.                       |


In [ ]:
GroupName = "testgroup12"  #Enter Group Name

Body = f"""{{
   "GroupID": "371b9766-1279-4d14-b435-901743cdb251",
   "GroupName":"{GroupName}"
}}"""
#GroupID: ID of the parent group, null means HOME

#Send request
response = requests.post(url+"group",auth=Credentials,data=Body,headers=headers)

print(response.text)
GroupID = response.text

41019308-df54-458f-9861-c69d9a52eefa


##Move Device to Group

**API to move devices to the group**

This api allows you to move all the Devices that are listed in List device section to the new group that has been created in Create Group section

Request Body Schema

| Key         | Data Type | Description                                |
|-------------|-----------|--------------------------------------------|
| DeviceId    | List     | ID of the device.                         |
| GroupId     | String     | The ID of the destination group for the device movement |


In [ ]:
Body = [
    {"DeviceId": device_id, "GroupId": GroupID} for device_id in DeviceIds
]

#Send request
response = requests.put(url + "deviceassignment", auth=Credentials, json=Body, headers=headers)
print("Move response:", response.text)

Move response: true


#User Management

##Create Device Group Set

**API to create Device Group set**

This api allows you to create Device Group set based on GroupIDs and Users assigned these Device Group Set will have the visibility of the devices which is in this device group set.

Request Body Schema

| Parameter                             | Data Type | Description                                           |
|---------------------------------|-----------|-------------------------------------------------------|
| TemplateName                    | String    | Name of the device set template.                     |
| TemplateDescription             | String    | Description of the device set template.              |
| TemplateData                    | List of string    | List of Group IDs             |
| isAllowedNewGroupOrJobFolder    | Integer   | Allow addition of new group according to need         |

Values for `isAllowedNewGroupOrJobFolder`:

| Value | Description                                               |
|-------|-----------------------------------------------------------|
| 0     | Don't Allow Any New Groups                               |
| 1     | Automatically Allow New Groups Added Within Home Group and Allowed Sub Groups |
| 2     | Automatically Allow New Groups Added Only Within Allowed Sub Groups |


In [ ]:
deviceSet_template_name = "GroupSet16"             #Name of Device Group Set
deviceSet_template_description = "Device_GroupSet13"
deviceSet_template_data = [GroupID]           #List of Device Group ID
isAllowedNewGroupOrJobFolder = 0
#isAllowedNewGroupOrJobFolder=> 0:Don't Allow Any New Groups, 1:Automatically Allow New Groups Added Within Home Group and Allowed Sub Groups, 2:Automatically Allow New Groups Added Only Within Allowed Sub Groups

# Construct the request body
Body = {
    "TemplateName": deviceSet_template_name,
    "TemplateDescription": deviceSet_template_description,
    "TemplateData": deviceSet_template_data,
    "isAllowedNewGroupOrJobFolder": isAllowedNewGroupOrJobFolder
}

# Send POST request
response = requests.post(url + "UserGroupPermission", auth=Credentials, json=Body, headers=headers)

# Print the response
print(json.dumps(json.loads(response.text), indent=4))

response_data = response.json()
for template in response_data:
    if template["TemplateName"] == deviceSet_template_name and template["TemplateDescription"]==deviceSet_template_description:
        GroupTemplateID = template["TemplateId"]
        print(GroupTemplateID)

[
    {
        "TemplateName": "! group disable",
        "TemplateData": null,
        "TemplateId": "00C14081-15B5-459B-9131-C3076DC109A1",
        "TemplateDescription": "",
        "isAllowedNewGroupOrJobFolder": 0,
        "isForClone": false,
        "folderType": 0,
        "ReportTemplateData": null,
        "JobFolderTemplateData": null,
        "DeviceGroupTemplateData": null,
        "RoleTemplateData": null,
        "RoleTemplateId": "0",
        "FolderTemplateId": "0",
        "GroupSetTemplateId": "0",
        "isOnlyTagAllowed": 0,
        "loginValidFrom": null,
        "loginValidTo": null
    },
    {
        "TemplateName": "sanity55",
        "TemplateData": null,
        "TemplateId": "03538AE0-E847-4FA7-B618-4A023287571A",
        "TemplateDescription": "",
        "isAllowedNewGroupOrJobFolder": 0,
        "isForClone": false,
        "folderType": 0,
        "ReportTemplateData": null,
        "JobFolderTemplateData": null,
        "DeviceGroupTemplateData": n

##Create Jobs/Profiles Folder Set

**API to create Jobs/profiles folder Set**

This api allows you to create Jobs/Profiles Folder and User assigned with this Jobs?Profiles Folder Set will have visibility of this Jobs/Profiles set only.

Request Body Schema

| Parameter                             | Data Type | Description                                           |
|---------------------------------|-----------|-------------------------------------------------------|
| TemplateName                    | String    | Name of the folder template.                         |
| TemplateDescription             | String    | Description of the folder template.                  |
| TemplateData                    | List of String    | List of Ids of Jobs/Profiles folder     |
| isAllowedNewGroupOrJobFolder    | Integer   | Allow addition of new groups or job folders according to need        |
| ReportTemplateData              | List of String    | List of Ids of Custom reports folder      |
| isForClone                      | Boolean   | Indicates if the template is for cloning purposes.  |

Values for `isAllowedNewGroupOrJobFolder`:

| Value | Description                                               |
|-------|-----------------------------------------------------------|
| 0     | Don't Allow Any New Folders                              |
| 1     | Automatically Allow New Folders Added Within Home Folder and Allowed Sub Folders |
| 2     | Automatically Allow New Folders Added Only Within Allowed Sub Folders |


In [ ]:
folder_template_name = "Create_Jobs/profiles_9"              #Name of Folder Set
folder_template_description = "Create_Jobs/profiles_9"
folder_template_data = [

]                                             #folder_template_data => List of Ids of Jobs/Profiles folder
is_allowed_new_group_or_job_folder = "0"
report_template_data = []                     #report_template_data => List of Ids of Custom reports folder
is_for_clone = False
#isAllowedNewGroupOrJobFolder=> 0:Don't Allow Any New Folders, 1:Automatically Allow New Folders Added Within Home Folder and Allowed Sub Folders, 2:Automatically Allow New Folders Added Only Within Allowed Sub Folders


Body = {
    "TemplateName": folder_template_name,
    "TemplateDescription": folder_template_description,
    "TemplateData": folder_template_data,
    "isAllowedNewGroupOrJobFolder": is_allowed_new_group_or_job_folder,
    "ReportTemplateData": report_template_data,
    "isForClone": is_for_clone
}

# Send POST request
response = requests.post(url + "UserJobPermission", auth=Credentials, json=Body, headers=headers)

print(json.dumps(json.loads(response.text), indent=4))

response_data = response.json()
for template in response_data:
    if template["TemplateName"] == folder_template_name and template["TemplateDescription"]==folder_template_description:
        JobsTemplateId = template["TemplateId"]

print(JobsTemplateId)


[
    {
        "TemplateName": "Create_Jobs/profiles_7",
        "TemplateData": null,
        "TemplateId": "0135403E-9438-4B12-A66B-EEDDAF036120",
        "TemplateDescription": "Create_Jobs/profiles_7",
        "isAllowedNewGroupOrJobFolder": 0,
        "isForClone": false,
        "folderType": 0,
        "ReportTemplateData": null,
        "JobFolderTemplateData": null,
        "DeviceGroupTemplateData": null,
        "RoleTemplateData": null,
        "RoleTemplateId": "0",
        "FolderTemplateId": "0",
        "GroupSetTemplateId": "0",
        "isOnlyTagAllowed": 0,
        "loginValidFrom": null,
        "loginValidTo": null
    },
    {
        "TemplateName": "ss",
        "TemplateData": null,
        "TemplateId": "08B2AA79-223A-47FC-86C1-1FD1075ADEC3",
        "TemplateDescription": "",
        "isAllowedNewGroupOrJobFolder": 0,
        "isForClone": false,
        "folderType": 0,
        "ReportTemplateData": null,
        "JobFolderTemplateData": null,
        "Devi

##Create Role

**Permission IDs and Description**
<details>
<summary><b style="font-size: 18px;">Click here to take the reference and add the value in Role_template_data</summary>

| Permission ID | Description                                  |
|---------------|----------------------------------------------|
| 1             | New Group                                    |
| 2             | Rename Group                                 |
| 3             | Delete Group                                 |
| 4             | Add/Delete Group Job                         |
| 5             | Delete Device                                |
| 6             | Apply Job                                    |
| 7             | Job Queue Remove job                         |
| 8             | Blacklist Device                             |
| 9             | Remote Device                                |
| 10            | Locate Device/Location Tracking              |
| 11            | Send Message To Device                       |
| 12            | Reboot Device                                |
| 13            | Lock Device                                  |
| 14            | Wipe Device                                  |
| 15            | Lock Applications                           |
| 16            | Uninstall Applications                       |
| 17            | Applicaiton Run At Atart Up                  |
| 18            | Monitor Application                          |
| 19            | Import Preapproved Device                    |
| 20            | Auto Approve Device                          |
| 21            | Approve Device                               |
| 22            | Whitelist Device                             |
| 23            | Close Ticket                                 |
| 24            | Delete Ticket                                |
| 25            | Add/Edit/Delete Service Centers              |
| 26            | Add/Edit/Delete Fault Codes                  |
| 27            | Delete Message                               |
| 28            | Reply Message                                |
| 29            | New Job                                      |
| 30            | Delete Job                                   |
| 31            | Modify Job                                   |
| 32            | Generate Configuration Barcodes              |
| 33            | System Log Report                            |
| 34            | Device Health Report                         |
| 35            | Device Connected Report                      |
| 36            | Jobs Deployed Report                         |
| 37            | Asset Tracking Report                        |
| 38            | Activity Console                             |
| 39            | Change Password                              |
| 40            | License Management                           |
| 41            | Add User                                     |
| 42            | Edit User                                    |
| 43            | Delete User                                  |
| 44            | Edit Device Name                             |
| 45            | Edit Device Notes                            |
| 46            | Assign Group                                 |
| 47            | CallLog Tracking Report                      |
| 48            | Call Logs                                    |
| 49            | SureLock Settings                            |
| 50            | SureFox Settings                             |
| 51            | App Version Tracking Report                  |
| 53            | Clear Application Data                       |
| 54            | SureVideo Settings                           |
| 55            | Device History Report                        |
| 56            | DataUsage Report                             |
| 57            | SmsLog Device                                |
| 58            | Allow Home Group                             |
| 59            | Edit Telecom Management                      |
| 60            | Allow new device group                       |
| 61            | Allow new Folder                             |
| 62            | Hide/Unhide Application                      |
| 63            | OS Platforms Chart                           |
| 64            | Available Battery Chart                      |
| 65            | Available RAM Chart                          |
| 66            | Available Storage Chart                      |
| 67            | Last Connected Chart                         |
| 68            | Online/Offline Chart                         |
| 69            | Device SIM Status Chart                      |
| 70            | Roaming Status Chart                         |
| 71            | Unread Mail Chart                            |
| 72            | Unapproved Devices Chart                     |
| 73            | Jobs Chart                                   |
| 74            | New folder                                   |
| 75            | Upload files                                 |
| 76            | Delete File/Folder                           |
| 77            | Add Android Profile                          |
| 78            | Edit Android Profile                         |
| 79            | Delete Android Profile                       |
| 80            | Set As Default Android Profile               |
| 81            | Enroll AFW Android Profile                   |
| 82            | Add iOS Profile                              |
| 83            | Edit iOS Profile                             |
| 84            | Delete iOS Profile                           |
| 85            | Set As Default iOS Profile                   |
| 86            | Settings Android Profile                     |
| 87            | Add Windows Profile                          |
| 88            | Edit Windows Profile                         |
| 89            | Delete Windows Profile                       |
| 90            | Set As Default Windows Profile               |
| 91            | Remote Buzz                                  |
| 92            | App Store Android Add                        |
| 93            | App Store Android Edit                       |
| 94            | App Store Android Remove                     |
| 95            | App Store iOS Add                            |
| 96            | App Store iOS Edit                           |
| 97            | App Store iOS Remove                         |
| 99            | Installed Job Report                         |
| 100           | Change Password                              |
| 101           | Enable or Disable User                       |
| 102           | Add macOS Profile                            |
| 103           | Edit macOS Profile                           |
| 104           | Delete macOS Profile                         |
| 105           | Set As Default macOS Profile                |
| 106           | App Store macOS Add                         |
| 107           | App Store macOS Edit                        |
| 108           | App Store macOS Remove                      |
| 109           | Export                                      |
| 112           | Clear History                               |
| 113           | Modify History                              |
| 114           | Export History                              |
| 115           | Apply Profile                               |
| 116           | Compliance Job Report                       |
| 117           | Device Activity Report                      |
| 118           | App Store Windows Add                       |
| 119           | App Store Windows Edit                      |
| 120           | App Store Windows Remove                    |
| 121           | Custom Report                               |
| 122           | View Custom Report                          |
| 123           | Modify Custom Report                        |
| 124           | Delete Custom Report                        |
| 125           | Allow Intercom                              |
| 126           | Remote Control                              |
| 127           | File Manager                                |
| 128           | Remote Clipboard                            |
| 129           | Task Manager                                |
| 130           | Data Usage Report                           |
| 131           | Branding Info                               |
| 132           | Device Enrollment Rules                     |
| 133           | EULA Disclaimer Policy                      |
| 134           | iOS/macOS Settings                          |
| 135           | Group Assignment Rules                      |
| 136           | SAML Single Sign-On                         |
| 137           | Alert Template                              |
| 138           | Customize Toolbar                           |
| 139           | Customize Nix/SureLock                       |
| 140           | Certificate Management                       |
| 141           | Data Analytics                               |
| 142           | Miscellaneous Settings                       |
| 143           | Mobile Email Management                      |
| 144           | Account Management                           |
| 145           | Manage Subscription                          |
| 146           | Over All Compliance Chart                    |
| 147           | OS Version Compliance Chart                  |
| 148           | Rooted Compliance Chart                      |
| 149           | Connectivity Compliance Chart                |
| 150           | Sim Change Compliance Chart                 |
| 151           | Password Policy Compliance Chart            |
| 152           | Battery Compliance Chart                    |
| 153           | Application Policy Compliance Chart         |
| 154           | Siem Integration                            |
| 155           | Chrome OS Device Management                 |
| 156           | Office365 Settings                          |
| 157           | Add Office365 Profile                       |
| 158           | Edit Office365 Profile                      |
| 159           | Delete Office365 Profile                    |
| 160           | Assign to groups Office365 Profile          |
| 161           | WebHooks Settings                           |
| 162           | Email Verification                          |
| 163           | Select unapproved Android Enterprise apps   |
| 164           | Plugins                                     |
| 165           | SD Card Change Compliance Chart             |
| 166           | Firmware Updates                            |
| 167           | Efota Registration Status Chart             |
| 168           | Update Firmware                             |
| 169           | Move to Folder Android Profile              |
| 170           | Move to Folder iOS Profile                  |
| 171           | Move to Folder Windows Profile              |
| 172           | Move to Folder macOS Profile                |
| 173           | Data Analytics Dashboard                    |
| 174           | Mobile Network Connectivity Compliance Chart|
| 175           | Windows Health Attestation                  |
| 176           | Intel AMT Status                            |
| 177           | Device Storage Compliance Chart             |
| 178           | Move Jobs To Folder                         |
| 179           | Set as Default QR Code                      |
| 180           | Modify QR Code                              |
| 181           | Delete QR Code                              |
| 182           | View QR Code                                |
| 183           | Create QR Code                              |
| 185           | Mark As Read                                |
| 186           | Mark As Unread                              |
| 187           | Cleanup Inbox                               |
| 188           | Add Linux Profile                           |
| 189           | Edit Linux Profile                          |
| 190           | Delete Linux Profile                        |
| 191           | Set As Default Linux Profile               |
| 192           | Move to Folder Linux Profile               |
| 193           | NAC Integration                            |
| 194           | Rename Dashboard                           |
| 195           | Copy Dashboard                             |
| 196           | Edit Dashboard                             |
| 197           | Delete Dashboard                           |
| 198           | My Favourite Dashboard                     |
| 199           | Dashboard Edit Layout                      |
| 200           | Dashboard Add Chart                        |
| 201           | Create Dashboard                           |
| 202           | Data Usage Status                          |
| 203           | Enterprise Integrations                    |
| 204           | Scan for threats                           |
| 205           | Windows Copy Genuine Validation            |
| 206           | Mobile Threat Defence                      |
| 221           | Create Custom Column                  |
| 222           | Delete Custom Column                  |
| 223           | Move Device                           |
| 224           | Security Patch Level                  |
| 225           | Custom                                |
| 226           | Windows Update                        |
| 256           | Edit Bluetooth Name                   |
| 257           | Modify Custom Column                  |
| 258           | Edit Android Enterprise               |
| 259           | Edit Geo Fence                        |
| 260           | Edit Time Fence                       |
| 261           | Edit Network Fence                    |
| 262           | Remove Installed Profile              |
| 263           | Reset Password                        |
| 264           | System Scan                           |
| 265           | Active Kiosk Application              |
| 266           | Wi-Fi Signal Strength                 |
| 267           | Location Access                       |
| 268           | Edit Custom Column                    |
| 269           | Things Management                     |
| 270           | Unapproved View                       |
| 271           | Preapproved View                            |
| 272           | Allow Enrollment                            |
| 273           | Block Enrollment                            |
| 274           | Invite User                                 |
| 275           | Blocklisted View                            |
| 276           | Pending Delete View                         |
| 277           | Force Delete                                |
| 278           | Jobs View                                   |
| 279           | Reports View                                |
| 280           | Dashboard View                              |
| 281           | Inbox View                                  |
| 283           | Profiles View                               |
| 284           | App Store View                              |
| 285           | Enrollment View                             |
| 286           | MTD View                                    |
| 287           | Allow FileStore Permissions                 |
| 288           | Device Encryption Report                    |
| 289           | Restore                                     |
| 290           | View Custom Property                        |
| 291           | Add Custom Property                         |
| 292           | Modify Custom Property                      |
| 293           | Delete Custom Property                      |
| 294           | Import Custom Property                      |
| 295           | View Custom property                        |
| 296           | Edit Custom property                        |
| 297           | Add Default Job                             |
| 298           | Delete Default Job                          |
| 301           | Retry Pending/Failed Jobs                   |
| 303           | Export Logs                                 |
| 304           | Device Logs                                 |
| 305           | Enable FileStore - all devices              |
| 306           | Allow Remote Screen Settings                |
| 307           | Allow Remote Screen Play/Pause              |
| 308           | Allow Remote Screen Swipe to Unlock         |
| 309           | Allow Remote Screen Volume                  |
| 310           | Allow Remote Screen Search                  |
| 311           | Allow Remote Screen Screenshot              |
| 312           | Allow Remote Screen Ctrl+alt+delete        |
| 313           | Allow Remote Screen Shortcut Management    |
| 314           | Allow Remote Screen Change View            |
| 315           | Allow Remote Screen Resolution Change      |
| 316           | Allow remote New Folder                    |
| 317           | Allow remote Delete                        |
| 318           | Allow remote Download                      |
| 319           | Allow remote Upload                        |
| 320           | Allow Remote Send to Device Clipboard      |
| 321           | Allow Remote Read from Device Clipboard    |
| 322           | Allow Remote Clear from Device Clipboard   |
| 323           | Allow Remote Kill Process Task Manager     |
| 324           | Allow Remote Kill AllTask Manager          |
| 325           | App Store Linux Add                        |
| 326           | App Store Linux Edit                       |
| 327           | App Store Linux Remove                     |
| 328           | Edit Things Info                           |
| 330           | Device Enrollment Restrictions             |
| 331           | Custom Properties                           |
| 332           | Import Custom property                      |
| 333           | Edit Custom Property                        |
| 334           | Group Details                               |
| 335           | Hardware Change                             |
| 336           | Device Encryption                           |
| 337           | Device Uptime                               |
| 338           | OS Build Number                             |
| 339           | Allow Remote View                           |
| 340           | Allow Remote Touch                          |
| 342           | Service Now                                 |
| 343           | Apps                                        |
| 344           | Power Off Device                            |
| 345           | Job Revision History                        |
| 346           | Revert Job                                  |
| 347           | Compare Job                                 |
| 348           | Save as Job                                 |
| 349           | Export Logs                                 |
| 350           | Download Logs                               |
| 351           | Request Report                              |
| 352           | OEM Configuration Status                    |
| 353           | Android Management                          |
| 358           | Play For Work                               |
| 360           | Add Custom Report                           |
| 361           | Add ChromeOS Profile                        |
| 362           | Edit ChromeOS Profile                       |
| 363           | Delete ChromeOS Profile                     |
| 364           | App Store ChromeOS Add                      |
| 365           | App Store ChromeOS Edit                     |
| 366           | App Store ChromeOS Remove                   |
| 367           | Apply Jobs/Profile                          |
| 1111          | Relay Server Report                         |
| 11001         | Auto fix rules                              |
| 12001         | Application version                         |
| 12002         | OS build version                            |

</details>


**API to create Role**

This api allows you to create Role based on specific sets of permission IDs and Users assigned these roles will have the ability to perform actions corresponding to the provided permission IDs. In other words, users will be limited to actions granted by the permissions associated with their roles.

Request Body Schema

| Parameter                | Data Type | Description                                |
|---------------------|-----------|--------------------------------------------|
| TemplateName        | String    | Name of the role template.                 |
| TemplateDescription | String    | Description of the role template.          |
| TemplateData        | List of Integer    | Set of permission ids      |


In [ ]:
Role_template_name = "licensed_india_role13"           #Name of Role Template
Role_template_description = "licensed_india_role13"       #Description of Role Template

Role_template_data = [1,2]
# Role_template_data contains IDs for specific permission.
# To add IDs, refer to the descriptions and include the corresponding values.

Body = {
    "TemplateName": Role_template_name,
    "TemplateDescription": Role_template_description,
    "TemplateData": Role_template_data
}

# Send POST request
response = requests.post(url + "UserRole", auth=Credentials, json=Body, headers=headers)

# Print the response
print(json.dumps(json.loads(response.text), indent=4))

response_data = response.json()
for template in response_data:
    if template["TemplateName"] == Role_template_name and template["TemplateDescription"]==Role_template_description:
        RoleTemplateID = template["TemplateId"]

print(RoleTemplateID)


[
    {
        "TemplateName": "32",
        "TemplateData": null,
        "TemplateId": "00E599B3-A118-4C12-9C0B-1615235AC994",
        "TemplateDescription": "",
        "isAllowedNewGroupOrJobFolder": 0,
        "isForClone": false,
        "folderType": 0,
        "ReportTemplateData": null,
        "JobFolderTemplateData": null,
        "DeviceGroupTemplateData": null,
        "RoleTemplateData": null,
        "RoleTemplateId": "0",
        "FolderTemplateId": "0",
        "GroupSetTemplateId": "0",
        "isOnlyTagAllowed": 0,
        "loginValidFrom": null,
        "loginValidTo": null
    },
    {
        "TemplateName": "licensed_india_role12",
        "TemplateData": null,
        "TemplateId": "04A3B2A6-35F4-4AAD-9CB1-93505D5708AD",
        "TemplateDescription": "licensed_india_role12",
        "isAllowedNewGroupOrJobFolder": 0,
        "isForClone": false,
        "folderType": 0,
        "ReportTemplateData": null,
        "JobFolderTemplateData": null,
        "Device

##Create Device Grid Column set

**Grid Column IDs and Description**
<details>
<summary><b style="font-size: 18px;">Click here to take the reference and add the value in Grid_data_template</summary>


| ID  | Description                   |
|-----|-------------------------------|
| 0   | Select All                    |
| 11  | Agent Version                 |
| 42  | Android Enterprise            |
| 57  | Android ID                    |
| 3   | Battery                       |
| 82  | Battery Serial Number         |
| 83  | Battery Charge Cycles         |
| 84  | Battery Health                |
| 33  | Bluetooth Status              |
| 40  | Bluetooth Name                |
| 35  | Bluetooth SSID                |
| 10  | Cell Signal Strength(%)       |
| 71  | Cell Signal Strength(dBm)     |
| 72  | CPU Temperature               |
| 22  | CPU USAGE                     |
| 49  | CTS Verified                  |
| 21  | Data Usage                    |
| 67  | DEP Server                    |
| 61  | Device ID                     |
| 63  | Device Locale                 |
| 55  | Device Local IP Address       |
| 7   | Device Registered             |
| 9   | Device Roaming                |
| 54  | Device Time Zone              |
| 30  | Device User Name              |
| 47  | Encryption Status             |
| 26  | Enrolled                      |
| 60  | Firmware Version              |
| 59  | FOTA Registration Status      |
| 39  | Free Program Memory           |
| 38  | Free Storage Memory           |
| 81   | Google Advertising ID            |
| 31   | GPS Status                       |
| 23   | GPU Usage                        |
| 43   | Group Path                       |
| 58   | Hash Code                        |
| 66   | Hostname                         |
| 16   | IMEI                             |
| 48   | IMEI2                            |
| 73   | IMSI                             |
| 80   | Installer Package                |
| 62   | Intel® AMT Status                |
| 4    | IP Address                       |
| 20   | KNOX License                     |
| 5    | Last Connected                   |
| 6    | Last Device Time                 |
| 65   | Last Logged In User              |
| 44   | Last System Scan                 |
| 88   | Manufacturer                     |
| 77   | Media performance Class          |
| 76   | MEID                             |
| 27   | Notes                            |
| 8    | Network Operator                 |
| 29   | Network Type                     |
| 37   | OS Build Number                  |
| 12   | OS Version                       |
| 17   | Phone Number                     |
| 50   | Platform Integrity               |
| 1    | Platform/Model                   |
| 28   | Polling Mechanism                |
| 69   | Processor                        |
| 19   | Device Privilege Status          |
| 41   | Security Path Date               |
| 18   | Serial Number                    |
| 32   | Sim serial Number(ICCID)         |
| 2    | Status                           |
| 25   | Supervised                       |
| 75   | SureDefence Version              |
| 14   | SureFox Version                  |
| 36   | SureLock Settings Identifier     |
| 13   | SureLock Version                 |
| 15   | SureVideo Version                |
| 24   | Battery Temperature(?)           |
| 45   | Threats Count                    |
| 51   | Threat Protection                |
| 53   | Unknown Source Apps              |
| 34   | USB Status                       |
| 52   | USB Debugging                    |
| 46   | Wi-Fi Hotspot Status             |
| 56   | WiFi SSID                        |
| 74   | WiFi Mac Address                 |
| 68   | WiFi Signal Strength(%)          |
| 70   | WiFi Signal Strength(dBm)        |
| 85   | Latitude                         |
| 86   | Longitude                        |
| 87   | Address                          |


</details>


**API to create Device grid Column Set**

This api allows you to create Device Grid Column based on specific sets of Grid column IDs and Users assigned these device grid column set will have the ability to perform actions corresponding to the provided Grid Column Ids.

Request Body Schema

| Key                 | Data Type | Description                                |
|---------------------|-----------|--------------------------------------------|
| TemplateName        | String    | Name of the grid template.                 |
| TemplateDescription | String    | Description of the grid template.          |
| DataTemplate        | List of Integer    | Set of grid column ids |


In [ ]:
Grid_template_name = "Create_DeviceGridColumnSet_8"               #Name of Grid Template
Grid_template_description = "Create_DeviceGridColumnSet_8"          #Descriptions of Grid Template
Grid_data_template = "[ 11,3]"
# Grid_data_template contains IDs for specific Column
# To add IDs, refer to the descriptions and include the corresponding values.

# Construct the request body
Body = {
    "TemplateName": Grid_template_name,
    "TemplateDescription": Grid_template_description,
    "DataTemplate": Grid_data_template
}

# Send POST request
response = requests.post(url + "devicegridcolumn/", auth=Credentials, json=Body, headers=headers)

# Print the response
print(json.dumps(json.loads(response.text), indent=4))

response_data = response.json()
for template in response_data:
    if template["TemplateName"] == Grid_template_name and template["TemplateDescription"]==Grid_template_description and template["DataTemplate"]==Grid_data_template:
        DeviceGridTemplateID = template["TemplateId"]

print(DeviceGridTemplateID)


[
    {
        "TemplateName": "!@#$%dfghffffff",
        "TemplateId": "08A81712-DE20-48CF-981C-ED1021F1D31D",
        "TemplateDescription": "",
        "DataTemplate": "[11,42,57,3,82,83,84,33,40,35,10,71,72,22,49,21,67,61,63,55,7,9,54,30,47,26,60,59,39,38,81,31,23,43,58,66,16,48,73,80,62,4,20,5,6,65,44,88,77,76,27,8,29,37,12,17,50,1,28,69,19,41,18,32,2,25,75,14,36,13,15,24,45,51,53,34,52,46,56,74,68,70,85,86,87]",
        "isForClone": false
    },
    {
        "TemplateName": "yada123qwer",
        "TemplateId": "08CEC6EE-0AB4-4CD1-B59B-E6A21397BDF0",
        "TemplateDescription": "",
        "DataTemplate": "[11,57,82,3,83,84]",
        "isForClone": false
    },
    {
        "TemplateName": "AllowAllColumn",
        "TemplateId": "0B50F147-4348-4D2F-BE24-F30C38C86404",
        "TemplateDescription": "",
        "DataTemplate": "[11,42,57,3,33,40,35,22,49,21,55,7,9,54,30,47,26,60,59,39,38,31,23,43,58,16,48,4,20,5,6,44,1,27,8,29,37,12,17,50,28,19,41,18,69,68,10,32,2,25,14,36,1

##Create User

**API to create Users**

This api allows you to create users with assigned variables such as GroupTemplateID, JobsTemplateId, RoleTemplateID, and DeviceGridTemplateID, which are predefined in their respective sections.Users will be limited to actions granted by the GroupTemplateID,JobsTemplateId,RoleTemplateID and DeviceGridTemplateID

Request Body Schema

| Parameter                     | Data Type | Description                                        |
|-------------------------|-----------|----------------------------------------------------|
| Password                | String    | Base64 encoded password                          |
| UserData                | Object    | User data including ID, name, email, and HideParentIfChildNotAllowed   |
| FeaturesTemplateId      | String        | The ID of the Role template to assign to the user         |
| GroupsTemplateId        | String       | The ID of the Device group set to assign to the user           |
| JobsTemplateId          | String       | The ID of the Jobs/Profiles folder to assign to the user    |
| Devicegridcolumntemplate| String       | The ID of the Device grid column set to assign to the user   |


In [ ]:
user_data = {
    "UserID": "Newuser17",
    "FirstName": "Newuser17",
    "LastName": "3",
    "Email": "Newuser4@getnada.com",
    "HideParentIfChildNotAllowed": True
}
#UserID:Enter the user name, FirstName: Enter first name of user, LastName: Enter the last name of user, Email:Enter the email

password = "Test@123"  #Enter the password
passwordEncoded = base64.b64encode(password.encode()).decode()  # Base64 encoded password
features_template_id = "673840CF-78D7-40B9-811E-0282F10B7494"                           #features_template_id:Role template id
groups_template_id = "945174F1-8ED0-49FE-BCC5-8C355508E2D1"                           #groups_template_id: Device group set id
jobs_template_id = "632C071D-0528-40C9-8489-C594EFDC86A7"                               #jobs_template_id: Jobs/Profiles folder set id
device_grid_column_template = "AE111F15-E5B0-411C-B277-2B7516C110AA"              #device_grid_column_template: Device grid column set id

#request body
Body = {
    "Password": passwordEncoded,
    "UserData": user_data,
    "FeaturesTemplateId": features_template_id,
    "GroupsTemplateId": groups_template_id,
    "JobsTemplateId": jobs_template_id,
    "Devicegridcolumntemplate": device_grid_column_template
}

# Send POST request
response = requests.post(url + "user", auth=Credentials, json=Body, headers=headers)

# Print the response
print(json.dumps(json.loads(response.text), indent=4))


{
    "Message": "User created successfully",
    "Data": null,
    "Result": true
}
